In [1]:
import pandas as pd
import requests
import numpy as np

In [2]:
#Set CSV file locations.  May need modification if data moved to other folder.
aqi_csvlocation = '../Datasets/aqi.csv'
asthma_csvlocation = '../Datasets/asthmaER.csv'
population_csvlocation = '../Datasets/PEP_2017_PEPANNRES_with_ann.csv'

#Read the CSV's into dataframes
aqidf = pd.read_csv(aqi_csvlocation)
asthmadf = pd.read_csv(asthma_csvlocation)
populationdf = pd.read_csv(population_csvlocation)

#Pull out only the aqi data about Georgia
gaaqidf = aqidf.loc[aqidf['State']=='Georgia']

#Set indices as counties for convenience
popdf = populationdf.set_index('GEO.display-label')
asthmadf = asthmadf.set_index('County')

#Pull out a list of the counties that we are actually pursuing
countylist = gaaqidf['County'].unique()

In [3]:
asthmadf

,2010,2011,2012,2013,2014,2015,2016,2017
County,,,,,,,,
Appling,58,59,74,78,68,53,71,61
Atkinson,37,16,45,42,60,32,29,28
Bacon,53,40,65,51,50,45,34,40
Baker,22,13,13,26,25,33,26,24
Baldwin,255,264,406,321,300,259,323,251
Banks,46,30,61,54,64,69,90,68
Barrow,319,284,333,381,369,452,386,473
Bartow,456,470,537,500,510,513,537,492
Ben Hill,118,82,115,131,179,154,120,141


In [4]:
#Remove commas and convert values to floats
#Otherwise we have Chatham County with values like 1,845

for year in np.arange(2010,2018):
    asthmadf[f'{year}'] = asthmadf[f'{year}'].str.replace(",","").astype(float)
asthmadf.loc['Chatham']

2010    1865.0
2011    1913.0
2012    2218.0
2013    2099.0
2014    1844.0
2015    1681.0
2016    1788.0
2017    1829.0
Name: Chatham, dtype: float64

In [5]:
#Saving as tempdf in order to keep from corrupting original dataframe
tempdf = asthmadf

In [6]:
#Adding new columns containing the prevalence per county per year

for year in np.arange(2010,2018):
    tempdf[f'{year} pcnt'] = [asthmadf.loc[county][f'{year}']/float(popdf.loc[f'{county} County, Georgia'][f'respop7{year}']) for county in asthmadf.index]

In [7]:
tempdf.head()

,2010,2011,2012,2013,2014,2015,2016,2017,2010 pcnt,2011 pcnt,2012 pcnt,2013 pcnt,2014 pcnt,2015 pcnt,2016 pcnt,2017 pcnt
County,,,,,,,,,,,,,,,,
Appling,58.0,59.0,74.0,78.0,68.0,53.0,71.0,61.0,0.003163,0.003195,0.004021,0.004240,0.003675,0.002873,0.003841,0.003294
Atkinson,37.0,16.0,45.0,42.0,60.0,32.0,29.0,28.0,0.004423,0.001911,0.005439,0.005069,0.007279,0.003821,0.003486,0.003357
Bacon,53.0,40.0,65.0,51.0,50.0,45.0,34.0,40.0,0.004790,0.003584,0.005816,0.004552,0.004457,0.003987,0.002992,0.003534
Baker,22.0,13.0,13.0,26.0,25.0,33.0,26.0,24.0,0.006405,0.003920,0.003850,0.007759,0.007587,0.010309,0.008105,0.007500
Baldwin,255.0,264.0,406.0,321.0,300.0,259.0,323.0,251.0,0.005581,0.005848,0.008731,0.006953,0.006543,0.005690,0.007148,0.005589


In [9]:
gaaqidf.head()

,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10
185,Georgia,Bibb,2017,365,284,81,0,0,0,0,93,58,40,0,0,163,0,202,0
186,Georgia,Charlton,2017,47,43,4,0,0,0,0,89,47,25,0,0,0,0,47,0
187,Georgia,Chatham,2017,365,289,76,0,0,0,0,84,57,38,0,0,100,35,230,0
188,Georgia,Chattooga,2017,240,232,8,0,0,0,0,64,45,36,0,0,240,0,0,0
189,Georgia,Clarke,2017,360,314,46,0,0,0,0,97,53,38,0,0,188,0,172,0


In [13]:
gaaqidf = gaaqidf.drop(columns='State')
gaaqidf = gaaqidf.set_index('County')

In [14]:
gaaqidf.describe()

,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10
count,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000
mean,2013.500000,292.165354,206.228346,82.803150,2.834646,0.283465,0.011811,0.003937,117.342520,61.771654,41.173228,0.031496,3.992126,114.759843,4.685039,168.165354,0.531496
std,2.283728,96.466117,72.973295,60.141012,4.490242,0.698928,0.108248,0.062746,37.740701,10.619312,6.747969,0.196290,18.845323,100.677279,17.375736,123.279641,2.671211
min,2010.000000,11.000000,10.000000,1.000000,0.000000,0.000000,0.000000,0.000000,52.000000,37.000000,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2012.000000,243.000000,169.500000,30.000000,0.000000,0.000000,0.000000,0.000000,93.000000,54.000000,37.000000,0.000000,0.000000,0.000000,0.000000,47.500000,0.000000
50%,2013.500000,351.000000,217.000000,73.000000,1.000000,0.000000,0.000000,0.000000,109.000000,61.000000,40.500000,0.000000,0.000000,115.000000,0.000000,189.000000,0.000000
75%,2015.000000,364.750000,259.500000,123.500000,4.000000,0.000000,0.000000,0.000000,146.500000,69.000000,45.000000,0.000000,0.000000,215.750000,0.000000,259.750000,0.000000
max,2017.000000,366.000000,347.000000,315.000000,33.000000,5.000000,1.000000,1.000000,364.000000,97.000000,64.000000,2.000000,121.000000,359.000000,99.000000,365.000000,22.000000


In [ ]:
#Next step: Keep days{year}, gooddays{year}, max{year}, median{year}, 90pcnt{year}